In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(0)  

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
scaler = MinMaxScaler()

train_df = pd.read_csv('assets/train.csv')
#test_df = pd.read_csv('assets/test.csv')
X = train_df.iloc[:, 0:-1]
y = train_df.iloc[:, -1]
y_1 = y.astype(int)

def training():
    X_train, X_test, y_train, y_test = train_test_split(X, y_1, random_state=0)
    
    lr = LogisticRegression().fit(X_train, y_train)
    
    #parameters = [{'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']},
              #{'penalty':['none', 'elasticnet', 'l1', 'l2']},
              #{'C':[0.001, 0.01, 0.1, 1, 10, 100]}]
    
    parameters = [{'penalty':['l1', 'l2']},{'C':[0.01,0.1, 10, 100]}]
        
    grid_search = GridSearchCV(estimator = lr,  
                               param_grid = parameters,
                               scoring = 'roc_auc')
    grid_search.fit(X_train, y_train)   

    print('Grid best parameter (max. {0}): {1}'.format('roc_auc', grid_search.best_params_))
    print('Grid best score ({0}): {1}'.format('roc_auc', grid_search.best_score_))
    
    best = grid_search.best_estimator_
    predicted = best.predict(X_test)

    print(classification_report(y_test, predicted, target_names=['not 1', '1']))
    
training()

Grid best parameter (max. roc_auc): {'penalty': 'l2'}
Grid best score (roc_auc): 0.8300367973991468
              precision    recall  f1-score   support

       not 1       0.91      0.99      0.95      2076
           1       0.65      0.15      0.24       234

    accuracy                           0.91      2310
   macro avg       0.78      0.57      0.60      2310
weighted avg       0.89      0.91      0.88      2310



In [3]:
def training2():
    X_train, X_test, y_train, y_test = train_test_split(X, y_1, random_state=0)
    
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    lr = LogisticRegression().fit(X_train_scaled, y_train)
    predicted = lr.predict(X_test_scaled)
    #print(X_train)
    print(classification_report(y_test, predicted, target_names=['not 1', '1']))

training2()

              precision    recall  f1-score   support

       not 1       0.91      0.99      0.95      2076
           1       0.58      0.11      0.18       234

    accuracy                           0.90      2310
   macro avg       0.74      0.55      0.56      2310
weighted avg       0.87      0.90      0.87      2310



In [ ]:
def engagement_model():
    lr = LogisticRegression().fit(X, y_1)
    res = lr.predict_proba(test_df)[:,1]
    ser = pd.Series(res, index=test_df.iloc[:, 0])
    
    return ser